In [66]:
from functools import reduce

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

SAMPLING_RATE = 20000.0
%matplotlib notebook

# Load sync data

In [76]:
sync_path = '../data/sync/event_list_tiempos-MR-0297.csv'
start_end_path = '../data/sync/start_end_frames_tiempos-MR-0297.txt'
repeated_path = '../data/sync/repeated_frames_tiempos-MR-0297.txt'
output_file = '../data/sync/sub_event_list_MR-0297_protocol_name.csv'

sync = pd.read_csv(sync_path)
start_end = np.loadtxt(start_end_path, dtype=int)
start_frame, end_frame = start_end[:, 0], start_end[:, 1]
repetead = np.loadtxt(repeated_path, dtype=int)


# Get event list for a specific protocol
Remember complete 'protocol_name' column in event_list_MR-XXX file to get a subset of event for a particular protocol.

In [77]:
protocol_name = 'chirp'  # name of protocol in event_list file

chirp_filter = sync['protocol_name'] == protocol_name
chirp_times = sync[chirp_filter]
chirp_times.head()


,n_frames,start_event,end_event,start_next_event,event_duration,event_duration_seg,inter_event_duration,inter_event_duration_seg,protocol_name,repetition_name,repeated_frames,#repeated_frames
1,2100,12278225,12981118,13061114,702893,35.14465,79996,3.99980,chirp,rep_000,[],0
2,2100,13061114,13764007,13844003,702893,35.14465,79996,3.99980,chirp,rep_001,[],0
3,2100,13844003,14546896,14626892,702893,35.14465,79996,3.99980,chirp,rep_002,[],0
4,2100,14626892,15329785,15409782,702893,35.14465,79997,3.99985,chirp,rep_003,[],0
5,2100,15409782,16112674,16192671,702892,35.14460,79997,3.99985,chirp,rep_004,[],0


## Set protocols time to split event in sub-events
For example in a chirp protocol each trial is a set of different protocols together
```
on pulse >> off pulse >> adaptation time >> frquency modulation >> adaptation time >> amplitude modulation >> adaptation time
```
then you need to define the order, time and name of these protocols.


In [80]:
fps = 60
names = ['ON', 'OFF', 'adap_0', 'FREQ', 'adap_1', 'AMP', 'adap_2']
times = [3, 3, 2, 15, 2, 8, 2]


# Get sub-events

In [81]:
start_trans = np.array([[0] + times[:-1]]).cumsum() * fps
end_trans = np.array(times).cumsum() * fps - 1

events_chirp = []
for kidx, kchirp in enumerate(chirp_times.itertuples()):
    rep_trans = np.zeros_like(times)
    sub_set = np.where(
            np.logical_and(
                start_frame >= kchirp.start_event,
                end_frame <= kchirp.end_event,)
        )[0]
    sub_start_frame = start_frame[sub_set]
    sub_end_frame = end_frame[sub_set]
    
    for ktrans, (start, end) in enumerate(zip(start_trans, end_trans)):
        rep_trans[ktrans] = np.logical_and(
            repetead >= sub_start_frame[start + rep_trans[:ktrans].sum()],
            repetead <= sub_end_frame[end + rep_trans[:ktrans].sum()]
        ).sum()
        
    idx = np.where(start_frame == kchirp.start_event)[0][0]
    start_event = start_frame[idx + start_trans + rep_trans]
    end_event = end_frame[idx + end_trans + rep_trans]
    df = pd.DataFrame(
        {
            'n_frames': end_trans - start_trans + rep_trans + 1,
            'start_event': start_event,
            'end_event': end_event,
            'start_next_event': end_event,
            'event_duration': end_event - start_event,
            'event_duration_seg': (end_event - start_event) / SAMPLING_RATE,
            'inter_event_duration': 0,
            'inter_event_duration_seg': 0.0,
            'protocol_name': protocol_name,
            'extra_desciption': names,
            'rep_name': kchirp.repetition_name,
            'repetead_frames': '',
            '#repetead_frames': rep_trans,
        })
    events_chirp.append(df)
events = reduce(lambda x, y: pd.concat([x, y]), events_chirp)    

# Save to csv

In [82]:
events.to_csv(output_file, sep=';', index=False)